In [63]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend

### Loading Dataset

In [64]:
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:, 3:13].values
y = df.iloc[:, 13].values

### Examining Data

In [65]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Creating dummy variables for countries and transforming to array

In [67]:
lbl_ncdr_x_1 = LabelEncoder()
X[:, 1] = lbl_ncdr_x_1.fit_transform(X[:, 1])
lbl_ncdr_x_2 = LabelEncoder()
X[:, 2] = lbl_ncdr_x_2.fit_transform(X[:, 2])
hot_ncdr = OneHotEncoder(categorical_features=[1])
X = hot_ncdr.fit_transform(X).toarray()
X = X[:, 1:]

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [68]:
X.shape

(10000, 11)

In [69]:
y.shape

(10000,)

### Creating train and test data

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Standardizing data

In [71]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

# Creating Classifier from Keras library

## Step 1: Create Function with layers

### Layer 1:

- using __add__ to add first hidden layer
- Adding 6 inputs, since our data has 11 columns
    - Using: ceil(# of columns/2) to get 6 layers
- Using rectifier function for hidden layers
- Input Dim is 11, since we have 11 variables and this is the first layer

### Layer 2:

- No need to add input dim, since it is not the first layer

### Layer 3:

- Using Sigmoid function for the output layer
- Since __y__ is binary, we only need one unit

## Step 2: Add ANN compile to the function

- Using 'adam' optimizer
- Using 'binary_crossentropy' __(log loss)__ to calculate loss, since it is a logistic regression
- Using 'accuracy' as metrics for accuracy

In [39]:
def _classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform',
                         activation='relu', input_dim=11))
    classifier.add(
        Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    
    classifier.compile(
        optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return classifier

### Running first model

In [40]:
classifier = _classifier()
# classifier.fit(X_train, y_train, batch_size=10, epochs=100)

In [41]:
# y_pred = classifier.predict(X_test)
# y_pred = (y_pred > 0.5)

In [42]:
# cm = confusion_matrix(y_test, y_pred)

In [43]:
print(f'Accuracy = {((1526+193)/20)}%')

Accuracy = 85.95%


## Creating k-fold  cross validation model

In [11]:
classifier = KerasClassifier(
    build_fn=cross_val_classifier, batch_size=10, epochs=100)

### Running K-fold cross validation model

In [14]:
accus = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)

Epoch 1/100
7200/7200 [==============================] - 2s 278us/step - loss: 0.4940 - acc: 0.7943
Epoch 2/100
7200/7200 [==============================] - 2s 217us/step - loss: 0.4208 - acc: 0.7946
Epoch 3/100
7200/7200 [==============================] - 2s 221us/step - loss: 0.4104 - acc: 0.8019
Epoch 4/100
7200/7200 [==============================] - 2s 226us/step - loss: 0.4011 - acc: 0.8268
Epoch 5/100
7200/7200 [==============================] - 2s 230us/step - loss: 0.3918 - acc: 0.8317
Epoch 6/100
7200/7200 [==============================] - 2s 223us/step - loss: 0.3847 - acc: 0.8337
Epoch 7/100
7200/7200 [==============================] - 2s 226us/step - loss: 0.3785 - acc: 0.8447
Epoch 8/100
7200/7200 [==============================] - 2s 223us/step - loss: 0.3739 - acc: 0.8487
Epoch 9/100
7200/7200 [==============================] - 2s 224us/step - loss: 0.3698 - acc: 0.8526
Epoch 10/100
7200/7200 [==============================] - 2s 225us/step - loss: 0.3673 - acc: 0.8503

### Accuracies from k-fold cross validation

In [15]:
accus

array([0.86624999, 0.82625   , 0.86124999, 0.86624999, 0.8275    ,
       0.8075    , 0.85499999, 0.85499999, 0.82125   , 0.84125   ])

### Taking the mean of all k-fold cross validation models to get an average from all tests

In [18]:
accu_mean = accus.mean()

In [21]:
accu_mean

0.8427499949559569

### Checking variance of all k-fold cross validation models

In [22]:
accu_var = accus.std()

In [23]:
accu_var

0.01989660622203791

## Adding __Dropout__ to k-fold  cross validation model

In [72]:
def dropout_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform',
                         activation='relu', input_dim=11))
    classifier.add(Dropout(rate=0.1))
    classifier.add(
        Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.1))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [73]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [74]:
parameters = {'batch_size':[33,,66, 99], 'nb_epoch': [99,198,496], 
              'optimizer': ['Adamax', 'rmsprop', 'sgd', '']}

In [85]:
grid_search = GridSearchCV(classifier, parameters, scoring = 'accuracy', cv=10)

In [86]:
grid_search = grid_search.fit(X=X_train, y=y_train)

Epoch 1/1
7200/7200 [==============================] - 3s 444us/step - loss: 0.5497 - acc: 0.7965
Epoch 1/1
7200/7200 [==============================] - 3s 428us/step - loss: 0.5468 - acc: 0.7974
Epoch 1/1
7200/7200 [==============================] - 3s 443us/step - loss: 0.5859 - acc: 0.7946
Epoch 1/1
7200/7200 [==============================] - 3s 450us/step - loss: 0.5835 - acc: 0.7937
Epoch 1/1
7200/7200 [==============================] - 3s 454us/step - loss: 0.5363 - acc: 0.7954
Epoch 1/1
7200/7200 [==============================] - 3s 460us/step - loss: 0.5656 - acc: 0.7931
Epoch 1/1
7200/7200 [==============================] - 3s 463us/step - loss: 0.5667 - acc: 0.7949
Epoch 1/1
7200/7200 [==============================] - 3s 467us/step - loss: 0.5506 - acc: 0.7946
Epoch 1/1
7200/7200 [==============================] - 3s 477us/step - loss: 0.5566 - acc: 0.7931
Epoch 1/1
7200/7200 [==============================] - 3s 482us/step - loss: 0.5475 - acc: 0.7953
Epoch 1/1
7200/7200 

In [87]:
best_params = grid_search.best_params_

In [88]:
best_accu = grid_search.best_score_

In [89]:
best_params

{'batch_size': 16, 'nb_epoch': 100, 'optimizer': 'Adamax'}

In [90]:
best_accu

0.795625

In [ ]:
accus = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)